<a href="https://colab.research.google.com/github/BINDHUPALLE/music_reccomendor/blob/main/music_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
with open("spotify.csv", 'r') as f:
    lines = f.readlines()
    print(lines[4230:4240])  # Print around the problematic line


['"\n', 'ABBA,The King Has Lost His Crown,/a/abba/the+king+has+lost+his+crown_20002945.html,"I believe your new girl turned you down  \n', "And they say she's pushin' you around  \n", 'How does it feel, I guess it hurts your pride  \n', "Tell me my friend how many nights you've cried  \n", '  \n', 'Disaster and disgrace  \n', 'The king has lost his crown  \n', 'Suddenly  \n', "He's clumsy like a clown  \n"]


In [ ]:
df = pd.read_csv("spotify.csv", engine='python', dtype=str)
df.head()


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
df.shape

(57650, 4)

In [ ]:
df.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [ ]:
#del duplicate
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

(57650, 4)

In [ ]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


#text cleaning


In [ ]:
df['text'] = df['text'].str.lower()
df['text'] = df['text'].replace(r'\n', ' ', regex=True)
df['text'] = df['text'].replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # remove punctuation


In [ ]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,look at her face its a wonderful face \r and ...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,take it easy with me please \r touch me gentl...
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,ill never know why i had to go \r why i had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,making somebody happy is a question of give an...


In [ ]:
df['artist'] = df['artist'].str.lower().str.replace(' ', '')


In [ ]:
df.head()

,artist,song,link,text
0,abba,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,look at her face its a wonderful face \r and ...
1,abba,"Andante, Andante",/a/abba/andante+andante_20002708.html,take it easy with me please \r touch me gentl...
2,abba,As Good As New,/a/abba/as+good+as+new_20003033.html,ill never know why i had to go \r why i had t...
3,abba,Bang,/a/abba/bang_20598415.html,making somebody happy is a question of give an...
4,abba,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,making somebody happy is a question of give an...


In [ ]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 16.5 MB/s eta 0:00:00


In [ ]:

CLIENT_ID = "70a9fb89662f4dac8d07321b259eaad7"
CLIENT_SECRET = "4d6710460d764fbbb8d8753dc094d131"

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_song_album_cover_url(song_name, artist_name):
    try:
        search_query = f"track:{song_name} artist:{artist_name}"
        results = sp.search(q=search_query, type="track")
        track = results["tracks"]["items"][0]
        album_cover_url = track["album"]["images"][0]["url"]
        spotify_url = track["external_urls"]["spotify"]
        return album_cover_url, spotify_url
    except:
        return None, None

In [ ]:
filtered_rows = []

for idx, row in df.sample(n=15000, random_state=42).iterrows():
    poster, url = get_song_album_cover_url(row['song'], row['artist'])
    if poster and url:
        row['album_cover_url'] = poster
        row['spotify_url'] = url
        filtered_rows.append(row)

# Create filtered DataFrame
filtered_music_df = pd.DataFrame(filtered_rows)

In [ ]:
filtered_music_df.head()

,artist,song,link,text,album_cover_url,spotify_url
44707,'nsync,Kiss Me At Midnight,/n/n+sync/kiss+me+at+midnight_20097440.html,kiss me at midnight \r 54321 \r kiss me at m...,https://i.scdn.co/image/ab67616d0000b273ad8819...,https://open.spotify.com/track/4mOPMSyc9tQsiB4...
51614,sia,Little Black Sandals,/s/sia/little+black+sandals_20662895.html,im being dragged down down by the hand \r the...,https://i.scdn.co/image/ab67616d0000b2732e6476...,https://open.spotify.com/track/5bcjREGkqwlYD27...
49532,queensryche,I Am I,/q/queensryche/i+am+i_20112781.html,beating with life you promised life \r securi...,https://i.scdn.co/image/ab67616d0000b273546ecb...,https://open.spotify.com/track/2YUgiPFiB17nAWh...
42071,loureed,Hold On,/l/lou+reed/hold+on_20085223.html,theres blacks with knives and whites with guns...,https://i.scdn.co/image/ab67616d0000b2736386bf...,https://open.spotify.com/track/7xIud6DM74zHh06...
21940,yello,Pinball Cha Cha,/y/yello/pinball+cha+cha_20148235.html,welcome \r in lucys bar \r hey you gentlemen...,https://i.scdn.co/image/ab67616d0000b27307df46...,https://open.spotify.com/track/1dNJI1TZxZAcDdD...


In [ ]:
filtered_music_df.to_csv("filtered_music.csv", index=False)


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df=pd.read_csv("filtered_music.csv")

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# STEP 1: Get unique artist embeddings
artist_embeddings = {}
for artist in df['artist'].unique():
    text = f"Artist: {artist}"
    vec = model.encode(text, convert_to_tensor=True)
    artist_embeddings[artist] = vec

# STEP 2: Compute combined embedding (lyrics + artist)
def get_combined_vector(row):
    lyric_vec = model.encode(row['text'], convert_to_tensor=True)
    artist_vec = artist_embeddings[row['artist']]
    combined_vec = (0.75*lyric_vec +0.25* artist_vec)  # or use concat for more complexity
    return combined_vec

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df['combined_vector'] = df.apply(get_combined_vector, axis=1)

# STEP 3: Convert all combined vectors into a matrix
embedding_matrix = torch.stack(df['combined_vector'].tolist()).cpu().numpy()

# STEP 4: Compute cosine similarity matrix
similarity_matrix = cosine_similarity(embedding_matrix)

In [ ]:
df.shape

(4490, 7)

In [ ]:
similarity_matrix.shape

(4490, 4490)

In [ ]:
import gzip
import pickle
import numpy as np
from tqdm import tqdm  # Optional: progress bar

top_n = 30
similarity_dict = {}

for idx in tqdm(range(len(df))):
    song_name = df.iloc[idx]['song']
    # Get sorted indices of top-N similar songs (excluding itself)
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_similar = sim_scores[1:top_n+1]  # Exclude itself at index 0

    similarity_dict[song_name] = [
        {
            "song": df.iloc[i]['song'],
            "artist": df.iloc[i]['artist'],
            "score": float(score)
        }
        for i, score in top_similar
    ]

# Save this much smaller structure
with gzip.open("top30_similar.pkl.gz", "wb") as f:
    pickle.dump(similarity_dict, f)


100%|██████████| 4490/4490 [00:26<00:00, 170.02it/s]


In [ ]:

def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity_matrix[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [ ]:
df.tail()

,artist,song,link,text,album_cover_url,spotify_url,combined_vector
4485,indigogirls,Free In You,/i/indigo+girls/free+in+you_10144409.html,a hard knock \r a cold clock \r ticking off ...,https://i.scdn.co/image/ab67616d0000b2739dd704...,https://open.spotify.com/track/6Urj1k5LHIAdGzk...,"[tensor(-0.0473), tensor(-0.0166), tensor(0.02..."
4486,beachboys,Do It Again,/b/beach+boys/do+it+again_20013891.html,its automatic when i \r talk with old friends...,https://i.scdn.co/image/ab67616d0000b27300342f...,https://open.spotify.com/track/0vhCcrN8ULryq1K...,"[tensor(-0.0236), tensor(-0.0143), tensor(0.04..."
4487,drake,The Search,/d/drake/the+search_21057247.html,feat saukrates \r \r intro saukrates \r th...,https://i.scdn.co/image/ab67616d0000b2731f475b...,https://open.spotify.com/track/6WwRKPpK1uPeDLE...,"[tensor(-0.0024), tensor(-0.0318), tensor(0.01..."
4488,everclear,Annabella's Song,/e/everclear/annabellas+song_20052064.html,i see you roll your eyes \r you know it makes...,https://i.scdn.co/image/ab67616d0000b27315ab84...,https://open.spotify.com/track/6aXonxYwurUI6Th...,"[tensor(-0.0374), tensor(0.0028), tensor(0.002..."
4489,nofx,Just The Flu,/n/nofx/just+the+flu_20101601.html,looking back to the past \r predictions of th...,https://i.scdn.co/image/ab67616d0000b273b53c62...,https://open.spotify.com/track/2soezOyDINe27Z3...,"[tensor(-0.0015), tensor(-0.0024), tensor(0.01..."
